# Protein Secondary Structure Prediction

## Part 1: Deep Learning with LSTM Networks

This notebook implements protein secondary structure prediction using two approaches:
1. **LSTM (Long Short-Term Memory) Networks** - Deep Learning approach
2. **Hidden Markov Models (HMM)** - Probabilistic approach with Baum-Welch algorithm

Author: Michele Copetti  
Course: Interdisciplinary Research Methods in Computational Biology  
Date: June 2024

## Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from itertools import groupby

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

## Data Preprocessing for LSTM

The preprocessing class handles:
- Filtering sequences without non-standard amino acids
- Parsing sequences to extract segments of target length
- One-hot encoding amino acids for LSTM input
- Converting labels to categorical format

In [ ]:
class Preprocessing:
    """
    Preprocessing pipeline for protein sequence data.
    
    Attributes:
        amino_acids (str): Standard 20 amino acids
        mapping_sst3 (dict): Mapping for 3-class secondary structure
        target_length (int): Fixed length for sequence segments
        secondary_type (str): Type of secondary structure classification ('sst3' or 'sst8')
    """
    
    def __init__(self, target_length, secondary_type='sst3'):
        self.amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
        self.mapping_sst3 = {'C': 0, 'H': 1, 'E': 2}  # Other: 0, Helix: 1, Sheet: 2
        self.mapping_sst8 = {'C': 0, 'H': 1, 'E': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}
        self.target_length = target_length
        self.secondary_type = secondary_type
        self.sequences = None
        self.labels = None

    def one_hot_encode(self, sequence):
        """
        Convert amino acid sequence to one-hot encoded tensor.
        
        Args:
            sequence (str): Amino acid sequence
            
        Returns:
            torch.Tensor: One-hot encoded representation (seq_len, 20)
        """
        indices = torch.tensor([self.amino_acids.index(aa) for aa in sequence], dtype=torch.long)
        return torch.nn.functional.one_hot(indices, num_classes=len(self.amino_acids)).type(torch.float)

    def process_labels(self, label):
        """
        Convert secondary structure labels to categorical indices.
        
        Args:
            label (str): Secondary structure label string
            
        Returns:
            torch.Tensor: Label indices
        """
        if self.secondary_type == 'sst3':
            return torch.tensor([self.mapping_sst3[char] for char in label], dtype=torch.long)
        elif self.secondary_type == 'sst8':
            return torch.tensor([self.mapping_sst8[char] for char in label], dtype=torch.long)
    
    def parser(self, labels, sequences):
        """
        Parse sequences to extract segments of target length with consistent labels.
        
        Args:
            labels (str): Secondary structure label sequence
            sequences (str): Amino acid sequence
            
        Returns:
            tuple: (list of sequence segments, list of corresponding labels)
        """
        # Group consecutive identical labels
        categories = [''.join(g) for _, g in groupby(labels)]
        
        splits = []
        corresponding_labels = []
        start = 0
        
        if categories:
            for category in categories:
                length = len(category)
                split = sequences[start:start+length]
                
                # Only keep segments of exact target length
                if len(split) == self.target_length:
                    splits.append(split)
                    corresponding_labels.append(category[0])  # Single label per segment
                    break  # One segment per protein sequence
                
                start += length
        
        return splits, corresponding_labels

    def preprocess_data(self, filepath, num_sequences=None):
        """
        Complete preprocessing pipeline for protein sequence dataset.
        
        Args:
            filepath (str): Path to CSV dataset
            num_sequences (int, optional): Number of sequences to process
            
        Returns:
            tuple: (sequence tensors, label tensors)
        """
        data = pd.read_csv(filepath)
        
        # Filter out sequences with non-standard amino acids
        filtered_data = data[data['has_nonstd_aa'] == False]

        if num_sequences:
            filtered_data = filtered_data.sample(n=num_sequences, random_state=42)

        # Parse sequences and labels
        labels = []
        sequences = []
        
        for lbl, seq in zip(filtered_data[self.secondary_type], filtered_data['seq']):
            sequence, label = self.parser(lbl, seq)
            if sequence:
                sequences.extend(sequence)
                labels.append(label[0])
        
        # Convert to tensors
        sequence_tensors = [self.one_hot_encode(seq) for seq in sequences]
        sequence_tensors = torch.stack(sequence_tensors).squeeze(0)
        label_tensors = self.process_labels(labels)

        self.sequences = sequence_tensors
        self.labels = label_tensors
        
        return sequence_tensors, label_tensors

## Load and Preprocess Data

**Important**: Update the `path` variable below with the location of your dataset.  
Dataset: [Protein Secondary Structure Dataset](https://www.kaggle.com/datasets/alfrandom/protein-secondary-structure)

In [ ]:
# Configuration
SECONDARY_OUTPUT_NUMBER = 3  # 3-class classification: Other, Helix, Sheet
TARGET_LENGTH = 7  # Typical length for alpha helices and beta sheets
NUM_SEQUENCES = 40000  # Number of sequences to process (set to None for all)

# Update this path to your dataset location
DATA_PATH = '/path/to/your/2018-06-06-ss.cleaned.csv'

# Initialize preprocessor
preprocessor = Preprocessing(target_length=TARGET_LENGTH, secondary_type='sst3')

# Preprocess data
sequence_tensors, label_tensors = preprocessor.preprocess_data(DATA_PATH, num_sequences=NUM_SEQUENCES)

print(f"Sequence tensor shape: {sequence_tensors.shape}")
print(f"Label tensor shape: {label_tensors.shape}")

## LSTM Model Definition

Our LSTM model consists of:
- Multiple stacked LSTM layers for hierarchical feature learning
- A fully connected layer for final classification
- Output dimension of 3 for three secondary structure classes

In [ ]:
class Model(nn.Module):
    """
    LSTM-based model for protein secondary structure prediction.
    
    Args:
        input_dim (int): Input feature dimension (20 for amino acids)
        hidden_dim (int): Hidden state dimension
        layer_dim (int): Number of LSTM layers
        output_dim (int): Output dimension (3 for 3-class classification)
    """
    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # LSTM layer: processes sequential input
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        
        # Fully connected layer: maps LSTM output to class probabilities
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        """
        Forward pass through the network.
        
        Args:
            x (torch.Tensor): Input sequences (batch_size, seq_len, input_dim)
            
        Returns:
            torch.Tensor: Class logits (batch_size, output_dim)
        """
        # LSTM forward pass
        output, (hn, cn) = self.lstm(x)
        
        # Take the last time step output for classification
        out = self.fc(output[:, -1, :])
        
        return out


class AminoAcidDataset(Dataset):
    """
    PyTorch Dataset wrapper for amino acid sequences.
    
    Args:
        sequences (torch.Tensor): Sequence data
        labels (torch.Tensor): Corresponding labels
    """
    
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

## Train-Validation-Test Split

Split the dataset:
- 60% for training
- 20% for validation
- 20% for testing

In [ ]:
# Calculate split sizes
total = len(sequence_tensors)
train_size = int(0.6 * total)
val_size = int(0.2 * total)
test_size = total - train_size - val_size

# Generate random permutation for splitting
indices = torch.randperm(total).tolist()

train_indices = indices[:train_size]
val_indices = indices[train_size:train_size + val_size]
test_indices = indices[train_size + val_size:]

# Create data splits
train_sequences = sequence_tensors[train_indices]
train_labels = label_tensors[train_indices]

val_sequences = sequence_tensors[val_indices]
val_labels = label_tensors[val_indices]

test_sequences = sequence_tensors[test_indices]
test_labels = label_tensors[test_indices]

# Create data loaders
train_dataset = AminoAcidDataset(train_sequences, train_labels)
val_dataset = AminoAcidDataset(val_sequences, val_labels)
test_dataset = AminoAcidDataset(test_sequences, test_labels)

trainloader = DataLoader(train_dataset, batch_size=100, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=100, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=100, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

## Model, Loss, and Optimizer Configuration

In [ ]:
# Initialize model
model = Model(
    input_dim=20,  # 20 amino acids
    hidden_dim=140,  # Hidden state size
    layer_dim=6,  # Number of LSTM layers
    output_dim=SECONDARY_OUTPUT_NUMBER  # 3 classes
)

# Loss function for multi-class classification
criterion = nn.CrossEntropyLoss()

# Adam optimizer with learning rate 10^-4
optimizer = optim.Adam(model.parameters(), lr=0.0001)

print(f"Model initialized with {sum(p.numel() for p in model.parameters())} parameters")

## Training Loop

Train the model for multiple epochs while monitoring both training and validation metrics.

In [ ]:
# Training configuration
NUM_EPOCHS = 175

# Track metrics
train_losses = []
val_losses = []
val_accuracies = []

# Training loop
for epoch in range(NUM_EPOCHS):
    # ==================== Training Phase ====================
    model.train()
    epoch_loss = 0
    train_correct = 0
    train_total = 0
    
    for sequences, labels in trainloader:
        optimizer.zero_grad()
        sequences = sequences.float()
        
        # Forward pass
        outputs = model(sequences)
        outputs_flat = outputs.view(-1, SECONDARY_OUTPUT_NUMBER)
        labels_flat = labels.view(-1)
        
        # Calculate loss
        loss = criterion(outputs_flat, labels_flat)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Track metrics
        epoch_loss += loss.item() * sequences.size(0)
        _, predicted = torch.max(outputs_flat, 1)
        train_correct += (predicted == labels_flat).sum().item()
        train_total += labels_flat.size(0)
    
    # Calculate training metrics
    epoch_loss /= len(trainloader.dataset)
    train_losses.append(epoch_loss)
    train_accuracy = train_correct / train_total
    
    # ==================== Validation Phase ====================
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for sequences, labels in valloader:
            sequences = sequences.float()
            outputs = model(sequences)
            outputs_flat = outputs.view(-1, SECONDARY_OUTPUT_NUMBER)
            labels_flat = labels.view(-1)
            
            # Calculate validation loss
            val_loss += criterion(outputs_flat, labels_flat).item() * sequences.size(0)
            
            # Calculate accuracy
            _, predicted = torch.max(outputs_flat, 1)
            val_correct += (predicted == labels_flat).sum().item()
            val_total += labels_flat.size(0)
    
    # Calculate validation metrics
    val_loss /= len(valloader.dataset)
    val_losses.append(val_loss)
    val_accuracy = val_correct / val_total
    val_accuracies.append(val_accuracy)

    # Print progress every 10 epochs
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}]')
        print(f'  Train Loss: {epoch_loss:.4f}, Train Acc: {train_accuracy:.4f}')
        print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}')

print("\nTraining complete!")

## Visualize Training Dynamics

In [ ]:
# Set plotting style
sns.set(style="whitegrid", context="talk")

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

# Plot losses
sns.lineplot(x=range(len(train_losses)), y=train_losses, ax=ax1, 
             label='Training Loss', color='blue', linewidth=2)
sns.lineplot(x=range(len(val_losses)), y=val_losses, ax=ax1, 
             label='Validation Loss', color='orange', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=14)
ax1.set_ylabel('Loss', fontsize=14)
ax1.set_title('Training and Validation Loss Over Epochs', fontsize=16)
ax1.legend(fontsize=12)
ax1.grid(True, which='both', linestyle='--', linewidth=0.5)

# Plot validation accuracy
sns.lineplot(x=range(len(val_accuracies)), y=val_accuracies, ax=ax2, 
             label='Validation Accuracy', color='green', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=14)
ax2.set_ylabel('Accuracy', fontsize=14)
ax2.set_title('Validation Accuracy Over Epochs', fontsize=16)
ax2.legend(fontsize=12)
ax2.grid(True, which='both', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

## Evaluate on Test Set

In [ ]:
# Evaluate model on test set
model.eval()
test_correct = 0
test_total = 0
all_labels = []
all_predictions = []

with torch.no_grad():
    for sequences, labels in testloader:
        outputs = model(sequences)
        outputs_flat = outputs.view(-1, SECONDARY_OUTPUT_NUMBER)
        labels_flat = labels.view(-1)
        
        # Get predictions
        _, predicted = torch.max(outputs_flat, 1)
        test_correct += (predicted == labels_flat).sum().item()
        test_total += labels_flat.size(0)
        
        # Store for classification report
        all_labels.extend(labels.numpy())
        all_predictions.extend(predicted.numpy())

# Calculate accuracy
test_accuracy = test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.4f}\n')

# Print detailed classification report
print("Classification Report:")
report = classification_report(
    all_labels, 
    all_predictions, 
    target_names=['Other (C)', 'Helix (H)', 'Sheet (E)']
)
print(report)

## Visualize Confusion Matrix

In [ ]:
# Compute confusion matrix
conf_matrix = confusion_matrix(all_labels, all_predictions)

# Normalize confusion matrix
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

# Plot normalized confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix_normalized, 
    annot=True, 
    fmt='.2f', 
    cmap="Blues", 
    xticklabels=['C', 'H', 'E'], 
    yticklabels=['C', 'H', 'E']
)
plt.xlabel("Predicted", fontsize=12)
plt.ylabel("True", fontsize=12)
plt.title("Normalized Confusion Matrix - LSTM Model", fontsize=14)
plt.show()

---

# Part 2: Hidden Markov Model Approach

This section implements protein secondary structure prediction using Hidden Markov Models (HMM) trained with the Baum-Welch algorithm.

## Data Preprocessing for HMM

HMMs use integer encoding instead of one-hot encoding.

In [ ]:
class PreprocessingHMM:
    """
    Preprocessing pipeline for HMM approach.
    Uses integer encoding instead of one-hot encoding.
    """
    
    def __init__(self, target_length, secondary_type='sst3'):
        self.amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
        self.mapping_sst3 = {'C': 0, 'H': 1, 'E': 2}
        self.mapping_sst8 = {'C': 0, 'H': 1, 'E': 2, 'B': 3, 'G': 4, 'I': 5, 'T': 6, 'S': 7}
        self.target_length = target_length
        self.secondary_type = secondary_type

    def encode_sequence(self, sequence):
        """Convert amino acid sequence to integer indices."""
        return np.array([self.amino_acids.index(aa) for aa in sequence], dtype=int)

    def process_labels(self, label):
        """Convert labels to integer indices."""
        if self.secondary_type == 'sst3':
            return np.array([self.mapping_sst3[char] for char in label])
        elif self.secondary_type == 'sst8':
            return np.array([self.mapping_sst8[char] for char in label])
    
    def parser(self, labels, sequences):
        """Parse sequences and labels."""
        categories = [''.join(g) for _, g in groupby(labels)]
        splits = []
        corresponding_labels = []
        start = 0
        
        for category in categories:
            length = len(category)
            split = sequences[start:start+length]
            if len(split) == self.target_length:
                splits.append(split)
                corresponding_labels.append(category[0])
                break
            start += length
            
        return splits, corresponding_labels

    def preprocess_data(self, filepath, num_sequences=None):
        """Complete preprocessing pipeline."""
        data = pd.read_csv(filepath)
        filtered_data = data[data['has_nonstd_aa'] == False]

        if num_sequences:
            filtered_data = filtered_data.sample(n=num_sequences, random_state=42)

        labels = []
        sequences = []
        
        for lbl, seq in zip(filtered_data[self.secondary_type], filtered_data['seq']):
            sequence, label = self.parser(lbl, seq)
            if sequence:
                sequences.extend(sequence)
                labels.extend(label)

        # Convert to arrays
        sequence_arrays = [self.encode_sequence(seq) for seq in sequences]
        label_arrays = self.process_labels(labels)
        
        return sequence_arrays, label_arrays

## Load and Preprocess Data for HMM

In [ ]:
# Preprocess data for HMM
preprocessor_hmm = PreprocessingHMM(target_length=7, secondary_type='sst3')
sequences, labels = preprocessor_hmm.preprocess_data(DATA_PATH, num_sequences=50000)

print(f"Number of sequences: {len(sequences)}")
print(f"Number of labels: {len(labels)}")

## Utility Function: Split Sequences by Class

In [ ]:
def split_by_class(sequences, labels):
    """
    Split sequences into three lists based on their class labels.
    
    Args:
        sequences (list): List of sequences
        labels (list): Corresponding labels
        
    Returns:
        tuple: (helix_sequences, sheet_sequences, other_sequences)
    """
    helix = []
    sheet = []
    other = []
    
    for sequence, label in zip(sequences, labels):
        if label == 1:  # Helix
            helix.append(sequence)
        elif label == 2:  # Sheet
            sheet.append(sequence)
        else:  # Other
            other.append(sequence)
    
    return helix, sheet, other

## Hidden Markov Model Implementation

Complete implementation of HMM with Baum-Welch training algorithm.

In [ ]:
class HMM:
    """
    Hidden Markov Model with Baum-Welch algorithm for training.
    
    Args:
        num_states (int): Number of hidden states
        num_symbols (int): Number of observation symbols (20 amino acids)
    """
    
    def __init__(self, num_states, num_symbols):
        self.num_states = num_states
        self.num_symbols = num_symbols
        self.epsilon = 1e-10  # Prevent log(0) errors

        # Initialize probabilities randomly with small epsilon
        self.transition_probs = np.random.rand(num_states, num_states) + self.epsilon
        self.emission_probs = np.random.rand(num_states, num_symbols) + self.epsilon
        self.initial_probs = np.random.rand(num_states) + self.epsilon

        # Normalize to ensure probabilities sum to 1
        self.transition_probs /= self.transition_probs.sum(axis=1, keepdims=True)
        self.emission_probs /= self.emission_probs.sum(axis=1, keepdims=True)
        self.initial_probs /= self.initial_probs.sum()

    def forward_algorithm(self, seq):
        """
        Forward algorithm: compute probability of observations up to time t.
        
        Args:
            seq (np.array): Observation sequence
            
        Returns:
            np.array: Forward probabilities (T, num_states)
        """
        T = len(seq)
        alpha = np.zeros((T, self.num_states))
        
        # Initialization
        for i in range(self.num_states):
            alpha[0][i] = self.initial_probs[i] * self.emission_probs[i][seq[0]]
        
        # Recursion
        for t in range(1, T):
            for j in range(self.num_states):
                alpha[t][j] = np.sum(
                    alpha[t-1] * self.transition_probs[:, j] * self.emission_probs[j][seq[t]]
                )
        
        return alpha
    
    def backward_algorithm(self, seq):
        """
        Backward algorithm: compute probability from time t+1 to end.
        
        Args:
            seq (np.array): Observation sequence
            
        Returns:
            np.array: Backward probabilities (T, num_states)
        """
        T = len(seq)
        beta = np.zeros((T, self.num_states))
        
        # Initialization
        beta[T - 1] = 1

        # Recursion (backward)
        for t in range(T-2, -1, -1):
            for i in range(self.num_states):
                beta[t][i] = np.sum(
                    self.transition_probs[i] * 
                    self.emission_probs[:, seq[t+1]] * 
                    beta[t+1]
                )
        
        return beta

    def expectation_step(self, seq):
        """
        E-step: Calculate expected state occupancy (gamma) and
        expected state transitions (xi).
        
        Args:
            seq (np.array): Observation sequence
            
        Returns:
            tuple: (xi, gamma)
        """
        T = len(seq)
        alpha = self.forward_algorithm(seq)
        beta = self.backward_algorithm(seq)

        # Calculate gamma: probability of being in state i at time t
        gamma = alpha * beta
        gamma /= gamma.sum(axis=1, keepdims=True)  # Normalize
        
        # Calculate xi: probability of transition from i to j at time t
        xi = np.zeros((T - 1, self.num_states, self.num_states))
        for t in range(T - 1):
            for i in range(self.num_states):
                for j in range(self.num_states):
                    xi[t][i][j] = (
                        alpha[t][i] * 
                        self.transition_probs[i][j] * 
                        self.emission_probs[j][seq[t+1]] * 
                        beta[t+1][j]
                    )
            xi[t] /= xi[t].sum()  # Normalize
        
        return xi, gamma

    def normalize_parameters(self):
        """Normalize all probability matrices to ensure they sum to 1."""
        self.transition_probs /= self.transition_probs.sum(axis=1, keepdims=True)
        self.emission_probs /= self.emission_probs.sum(axis=1, keepdims=True)
        self.initial_probs /= self.initial_probs.sum()

    def train(self, sequences, num_iterations=1000):
        """
        Train HMM using Baum-Welch algorithm.
        
        Args:
            sequences (list): List of observation sequences
            num_iterations (int): Number of EM iterations
        """
        R = len(sequences)  # Number of sequences
        
        for iteration in range(num_iterations):
            # Initialize accumulators for M-step
            total_xi_probs = np.zeros_like(self.transition_probs)
            gamma_indicator_probs = np.zeros_like(self.emission_probs)
            total_gamma_probs = np.zeros((self.num_states, 1))
            total_initial_probs = np.zeros_like(self.initial_probs)

            # E-step: Calculate expectations for all sequences
            for seq in sequences:
                xi, gamma = self.expectation_step(seq)
                
                # Accumulate statistics
                total_xi_probs += xi.sum(axis=0)
                total_initial_probs += gamma[0]
                total_gamma_probs += gamma[:-1].sum(axis=0).reshape(-1, 1)
                
                # Accumulate emission statistics
                for t in range(len(seq)):
                    symbol = seq[t]
                    gamma_indicator_probs[:, symbol] += gamma[t]

            # M-step: Update parameters
            self.transition_probs = total_xi_probs / total_gamma_probs
            self.emission_probs = gamma_indicator_probs / total_gamma_probs
            self.initial_probs = total_initial_probs / R
            
            # Ensure probabilities sum to 1
            self.normalize_parameters()
            
            # Print progress
            if (iteration + 1) % 100 == 0:
                print(f"  Iteration {iteration + 1}/{num_iterations} complete")

    def score(self, seq):
        """
        Calculate log-likelihood of sequence under this model.
        
        Args:
            seq (np.array): Observation sequence
            
        Returns:
            float: Log-likelihood
        """
        alpha = self.forward_algorithm(seq)
        return alpha[-1].sum()

## Train and Evaluate HMM Models

In [ ]:
def train_and_evaluate_hmm(sequences, labels, n_iter=1000):
    """
    Train three HMMs (one per class) and evaluate on test set.
    
    Args:
        sequences (list): List of sequences
        labels (list): Corresponding labels
        n_iter (int): Number of training iterations
    """
    # Split data into train and test
    training, testing, train_labels, test_labels = train_test_split(
        sequences, labels, test_size=0.2, random_state=42
    )

    # Split training data by class
    helix, sheet, other = split_by_class(training, train_labels)
    
    print(f"Training set sizes:")
    print(f"  Helix: {len(helix)}")
    print(f"  Sheet: {len(sheet)}")
    print(f"  Other: {len(other)}\n")

    # Create and train three HMMs
    print("Training Helix HMM...")
    hmm_helix = HMM(num_states=3, num_symbols=20)
    hmm_helix.train(helix, num_iterations=n_iter)
    
    print("\nTraining Sheet HMM...")
    hmm_sheet = HMM(num_states=3, num_symbols=20)
    hmm_sheet.train(sheet, num_iterations=n_iter)
    
    print("\nTraining Other HMM...")
    hmm_other = HMM(num_states=3, num_symbols=20)
    hmm_other.train(other, num_iterations=n_iter)

    # Prediction
    models = [hmm_other, hmm_helix, hmm_sheet]  # Order: 0, 1, 2
    prediction_labels = [0, 1, 2]  # Other, Helix, Sheet

    print("\nEvaluating on test set...")
    correct = 0
    total = len(testing)
    all_true_labels = []
    all_pred_labels = []

    # Predict for each test sequence
    for sequence, true_label in zip(testing, test_labels):
        # Calculate likelihood under each model
        log_probs = [model.score(sequence) for model in models]
        
        # Predict class with highest likelihood
        best_model_idx = np.argmax(log_probs)
        predicted_label = prediction_labels[best_model_idx]

        all_true_labels.append(true_label)
        all_pred_labels.append(predicted_label)

        if true_label == predicted_label:
            correct += 1

    # Calculate metrics
    accuracy = (correct / total) * 100
    print(f'\nAccuracy: {accuracy:.2f}%\n')

    # Print classification report
    target_names = ['Other', 'Helix', 'Sheet']
    print("Classification Report:")
    report = classification_report(all_true_labels, all_pred_labels, target_names=target_names)
    print(report)

    # Plot confusion matrix
    conf_matrix = confusion_matrix(all_true_labels, all_pred_labels)
    conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(8, 6))
    sns.heatmap(
        conf_matrix_normalized, 
        annot=True, 
        fmt='.2f', 
        cmap="Blues", 
        xticklabels=target_names, 
        yticklabels=target_names
    )
    plt.xlabel("Predicted", fontsize=12)
    plt.ylabel("True", fontsize=12)
    plt.title("Normalized Confusion Matrix - HMM Model", fontsize=14)
    plt.show()


# Train and evaluate
train_and_evaluate_hmm(sequences, labels, n_iter=1000)

---

## Summary and Conclusions

This notebook demonstrated two approaches for protein secondary structure prediction:

### LSTM Approach
- **Accuracy**: ~88%
- **Strengths**: High accuracy, captures long-range dependencies
- **Limitations**: Black-box model, computationally expensive

### HMM Approach
- **Accuracy**: ~76%
- **Strengths**: Interpretable, computationally efficient, theoretically grounded
- **Limitations**: Lower accuracy, restrictive Markov assumptions

### Key Takeaways
1. Deep learning (LSTM) provides superior predictive accuracy
2. HMMs offer interpretability and computational efficiency
3. Both approaches are viable depending on the application requirements
4. The choice depends on the trade-off between accuracy and interpretability

For detailed analysis and discussion, please refer to the accompanying report.